<a href="https://colab.research.google.com/github/robgon-art/fan-fabler/blob/main/FanFabler_Llama_3_Inference_with_Unsloth_and_Wiki_Reference_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Apr 30 20:03:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   49C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch
max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "robgonsalves/llama-3-8b-Instruct-bnb-4bit-lora-64-64-lora",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import requests
import time

def get_snippets(wp_code, subject, limit=5):
    search_url = f"https://{wp_code}.wikipedia.org/w/api.php"
    search_params = {
        "action": "opensearch",
        "search": subject,
        "limit": str(limit),
        "namespace": "0",
        "format": "json"
    }

    search_response = requests.get(search_url, params=search_params)
    search_data = search_response.json()

    titles = search_data[1]
    results = []
    for title in titles:
        extracts_params = {
            "action": "query",
            "format": "json",
            "prop": "extracts",
            "exsentences": 20,  # Estimate for 500 words
            "titles": title,
            "explaintext": True,
        }

        extracts_response = requests.get(search_url, params=extracts_params)
        extracts_data = extracts_response.json()

        pages = extracts_data['query']['pages']
        for page_id in pages:
            page = pages[page_id]
            snippet = page.get('extract', 'Content not available.').replace('\n', ' ')
            results.append({"title": title, "snippet": snippet})

    return results


In [ ]:
system_prompt = """Respond to the user's query based on the conversation and the context.
Respond in the same language as the user's query.
To get more from context from the Wikipedia, indicate a Wiki page and language code at the end of the repsonse using this format:
>>>search term
>>>language code"""

single_message_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{}<|begin_of_text|><|start_header_id|>user<|end_header_id|>
{}<|begin_of_text|><|start_header_id|>assistant<|end_header_id|>
"""

FastLanguageModel.for_inference(model)

In [ ]:
first_question = "List three ideas for writing fan fiction about Curb Your Enthusiasm."

text = single_message_prompt.format(system_prompt, first_question)
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
resulting_tokens = model.generate(**inputs, streamer = text_streamer,
  max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id, do_sample = True)

Here are three ideas for writing fan fiction about Curb Your Enthusiasm:

1. A story centered around Larry David's interactions with a new character who is a die-hard fan of the show, creating humorous misunderstandings and awkward situations.
2. Exploring what would happen if Larry found himself on the receiving end of a misunderstanding similar to those he causes, and navigating the consequences of being the one at the center of chaos.
3. A romantic fan fiction story focusing on Larry's relationships, perhaps involving a new love interest that challenges his social norms and creates unexpected romantic misadventures.

>>>Curb Your Enthusiasm
>>>en<|eot_id|>


In [ ]:
resulting_text = tokenizer.batch_decode(resulting_tokens)
results_list = resulting_text[0].split("<|end_header_id|>")
first_answer = results_list[-1].replace("<|eot_id|>", "")
# print(first_answer)
results_list = first_answer.split(">>>")
search_term = results_list[-2].replace("search term:", "").strip()
language_code = results_list[-1].replace("language code:", "").strip()
print(search_term, " - ", language_code)

Curb Your Enthusiasm  -  en


In [ ]:
supporting_text = ""

if search_term and language_code:
  supporting_text_array = get_snippets(language_code, search_term)
  if len(supporting_text_array) > 1:
    # print(supporting_text_array)
    supporting_text = supporting_text_array[0]["snippet"]
    support_text = supporting_text_array[0]
    supporting_text = supporting_text.replace("\n", " ")

print(supporting_text)

In [ ]:
second_question = "Write an outline for The Talent Show Takedown"

long_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{}<|eot_id|><|start_header_id|>user<|end_header_id|>
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}<|eot_id|><|start_header_id|>user<|end_header_id|>
{}<|eot_id|><|start_header_id|>user<|end_header_id|>
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}"""

text = long_prompt.format(system_prompt, first_question, first_answer, supporting_text, second_question, "")
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
resulting_tokens = model.generate(**inputs, streamer = text_streamer,
  max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id, do_sample = True)

The Talent Show Takedown is a fan fiction episode of Curb Your Enthusiasm, where Larry David gets involved in a school talent show for his children's benefit. The story revolves around Larry trying to outdo the other parents by using a high-tech video production for his son's act, which leads to a series of mishaps and misunderstandings. As Larry navigates the competitive atmosphere and his own ego, the episode becomes a hilarious exploration of social awkwardness, personal growth, and fatherly love.  Characters involved:  1. Larry David - The main character who gets caught up in the talent show and tries to outshine the other parents. 2. Cheryl - Larry's wife who tries to reason with him and prevent the chaos from unfolding. 3. Jeff Greene - Larry's friend and manager who offers unsolicited advice and meddling in the situation. 4. The other parents - Various characters who participate in the talent show and clash with Larry.  This episode delves into the themes of family dynamics, amb

In [ ]:
prompt = "Citez trois idées pour écrire une fanfiction sur Amélie."
text = single_message_prompt.format(system_prompt, prompt)
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
resulting_tokens = model.generate(**inputs, streamer = text_streamer,
  max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id, do_sample = True)

Bien sûr! Voici trois idées pour écrire une fanfiction sur Amélie:
1. Amélie tombe amoureuse d'un mystérieux personnage introduit dans le film. Ils commencent à se fréquenter en secret, et Amélie doit faire face aux défis de l'amour tout en gardant son identité cachée.
2. Alors que Nino quitte Amélie, elle se retrouve dévastée. Son fanfiction pourrait explorer le déroulement de ce départ, et son influence sur Amélie qui essaye de continuer à faire des bonheurs sans Nino.
3. Amélie se retrouve impliquée dans une intrigue intrigante où elle doit relever un défi particulier pour sauver un personnage aimé. Ce défi développerait ses compétences et sa résilience pour accomplir son premier bonheur à elle-même.

>>>Amélie
>>>fr<|eot_id|>


In [ ]:
prompt = "방탄소년단에 관한 팬픽션 아이디어 3가지를 보여주세요."
text = single_message_prompt.format(system_prompt, prompt)
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
resulting_tokens = model.generate(**inputs, streamer = text_streamer,
  max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id, do_sample = True)

방탄소년단에 관한 팬픽션 아이디어를 몇 가지 제시해 드리겠습니다. 

1. 미스터 션샤인 TV 드라마와의 차가운 팀업 스토리: 방탄소년단 멤버들이 드라마 '미스터 션샤인' 속의 캐릭터들과 팀업하여 무대에서 함께 노래하는 판타지 스토리입니다. 

2. 영화 '히어로 프로젝트' 속에 오브젝트를 중심으로하는 Fan Fiction: 방탄소년단 멤버들이 히어로 프로젝트 세계에서 등장하며 각자의 모습을 보여주는 다양한 스토리들을 중심으로 한 Fan Fiction를 만드는 것입니다.

3. '이방인' 소설의 패러디: 방탄소년단 멤버들이 '이방인' 소설의 세계 속에서 대학교를 다니는 학생으로 등장하면서 어떻게 펼쳐질 것인지를 상상해 보는 것입니다.

이러한 아이디어를 바탕으로 방탄소년단 팬픽션을 쓰는 것은 분명히 재미있을 것이라 생각합니다! 더 많은 아이디어가 필요하신가요? 
>>>방탄소년단
>>>ko<|eot_id|>


In [ ]:
prompt = "팬픽션이란 무엇인가요? 한 문장으로 대답하세요. 한국어로 답장해주세요."
text = single_message_prompt.format(system_prompt, prompt)
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
resulting_tokens = model.generate(**inputs, streamer = text_streamer,
  max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id, do_sample = True)

팬픽션은 특정 작품의 지인들과 팬들을 대상으로 쓴 창작물로, 원작의 캐릭터나 설정을 바탕으로 새로운 이야기를 만들어주는 것입니다. 
>>>fan fiction
>>>ko<|eot_id|>
